<a href="https://colab.research.google.com/github/kimdonggyu2008/tacotron_tts_practice/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B3%B5%EB%B0%B1_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
from tqdm.notebook import tqdm
import librosa
from pathlib import Path
import matplotlib.pyplot as plt
import IPython.display as ipd
import glob
import soundfile as sf

In [ ]:
def load_audio(file_path, sr=22050):
    """
       file_path : 파일위치
       sr : 오디오를 읽을 때 Sampling rate 지정
    """
    ## 확장자 추출
    ext = Path(file_path).suffix

    ## 파일 읽기
    if ext in ['.wav', '.flac']:
        wav, sr = librosa.load(file_path, sr=sr)
    elif ext == '.pcm':
        wav = np.memmap(file_path, dtype='h', mode='r').astype('float32') / 32767
    elif ext in ['.raw', '.RAW']:
        wav, sr = sf.read(file_path, channels=1, samlerate=sr, format='RAW', subtype='PCM_16')
    else:
        raise ValueError("Unsupported preprocess method : {0}".format(ext))

    return wav, sr



In [ ]:
def trim_audio(wav,top_db=10,pad_len=4000):
  non_silence_indices=librosa.effects.split(wav,top_db=top_db)
  start=non_silence_indices[0][0]
  end=non_silence_indices[-1][1]

  wav=wav[start:end]
  wav=np.hstack([np.zeros(pad_len),wav,np.zeros(pad_len)])

  return wav


In [ ]:
def plot_wav(wav, sr):
    ## 그려보기
    plt.figure(1)

    plot_a = plt.subplot(211)
    plot_a.plot(wav)
    plot_a.set_xlabel('sample rate * time')
    plot_a.set_ylabel('energy')

    plot_b = plt.subplot(212)
    plot_b.specgram(wav, NFFT=1024, Fs=sr, noverlap=900)
    plot_b.set_xlabel('Time')
    plot_b.set_ylabel('Frequency')

    plt.show()

In [ ]:
## 시작하기

## 타코트론2는 기본적으로 22050 sampling rate에서 동작
sampling_rate = 22050
## 개인설정에 따라 특정 소리보다 작은 음성을 삭제하도록 설정
decibel=10

## Wav 파일 읽어오기  pcm 또는 다른 확장자도 사용 가능.
root_path = '/content/drive/MyDrive/코딩공부/Tacotron2-Wavenet-Korean-TTS-master/datasets/woman/audio'
file_list = glob.glob(os.path.join(root_path, "*.wav"))
#file_list = glob.glob(os.path.join(root_path, "*.pcm"))

## 저장할 위치 선택
save_path = '/content/drive/MyDrive/코딩공부/Tacotron2-Wavenet-Korean-TTS-master/datasets/woman/audio_edit'
os.makedirs(save_path, exist_ok=True)

for file_path in tqdm(file_list):

    ## 파일 불러오기(타코트론2는 기본적으로 22050 sampling rate에서 동작)
    wav, sr = load_audio(file_path, sr=sampling_rate)

    ## 오디오 자르기(패딩 추가)
    trimed_wav= trim_audio(wav, top_db=decibel)

    filename=Path(file_path).name
    temp_save_path = os.path.join(save_path, filename)

    ## 저장하기
    sf.write(temp_save_path, trimed_wav, sampling_rate)

  0%|          | 0/41 [00:00<?, ?it/s]